# Laboratorium 4
### Michał Kurleto

In [1]:
import sys
from scipy import sparse
import os
import re
import numpy as np
import nltk
import math
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Zadanie 1. Wyszukiwarka
### 1. Przygotowanie danych
Przygotuj duży (> 1000 elementów) zbiór dokumentów tekstowych w języku angielskim (np. wybrany korpus tekstów, podzbiór artykułów Wikipedii, zbiór dokumentów HTML uzyskanych za pomocą Web crawlera, zbiór rozdziałów wyciętych z
różnych książek)



Pliki stworzone z podzielenia książek "Moby-Dick" i "Pride and Prejudice" na około 500 plikow txt każda.

In [2]:
files = os.listdir('books/')
len(files)

1012

Przykładowy plik

In [3]:
with open('books/moby_dick_pt_10.txt','r',encoding="utf8") as file:
    content = file.read()
    print(content)

  “Scarcely had we proceeded two days on the sea, when about sunrise a
  great many Whales and other monsters of the sea, appeared. Among the
  former, one was of a most monstrous size.... This came towards us,
  open-mouthed, raising the waves on all sides, and beating the sea
  before him into a foam.” —_Tooke’s Lucian_. “_The True History_.”




  “He visited this country also with a view of catching horse-whales,
  which had bones of very great value for their teeth, of which he
  brought some to the king.... The best whales were catched in his own
  country, of which some were forty-eight, some fifty yards long. He
  said that he was one of six who had killed sixty in two days.”
  —_Other or Other’s verbal narrative taken down from his mouth by King
  Alfred, A.D._ 890.

  “And whereas all the other things, whether beast or vessel, that
  enter into the dreadful gulf of this monster’s (whale’s) mouth, are
  immediately lost and swallowed up, the sea-gudgeon retires into it in
  gr

### 2.
Określ słownik słów kluczowych (termów) potrzebny do wyznaczenia wektorów
cech bag-of-words (indeksacja). Przykładowo zbiorem takim może być unia wszystkich słów występujących we wszystkich tekstach.


In [4]:
def create_dictionary(files):
    dictionary = {}
    
    for file in files:
        with open('books/' + file,'r',encoding="utf8") as file_obj:
            contents = file_obj.read()
            
            words = nltk.word_tokenize(contents)
            
            #remove all punctuation
            words = [word.lower() for word in words if word.isalnum()]
            
            for word in words:
                if word in dictionary:
                    dictionary[word] += 1
                else:
                    dictionary[word] = 1
                    
    return dictionary.keys(), dictionary

20 najpopularniejszych słów:

In [5]:
dictionary, word_quantities = create_dictionary(files)
words = sorted(word_quantities.items(), key = lambda x: x[1], reverse=True)
print("Liczba słów w słowniku: {0}\n".format(len(words)))
print("20 najpopularniejszych słów: \n")
for i, word in enumerate(words):
    if i == 20: break
    print(word[0] + ': ' + str(word[1]))

Liczba słów w słowniku: 18706

20 najpopularniejszych słów: 

the: 19088
of: 10425
and: 10054
to: 8864
a: 6694
in: 6125
that: 4559
i: 4101
it: 3982
his: 3767
was: 3473
he: 3190
as: 2918
with: 2862
not: 2772
but: 2760
for: 2700
is: 2615
her: 2507
s: 2442


### 3. Vector
Dla każdego dokumentu j wyznacz wektor cech bag-of-words dj zawierający częstości występowania poszczególnych słów (termów) w tekście.

In [6]:
def create_vector(file, dictionary):
    vector = {}
    
    for word in dictionary:
        vector[word] = 0
    
    with open('books/' + file,'r',encoding="utf8") as file_obj:
        contents = file_obj.read()

    words = nltk.word_tokenize(contents)

    #remove all punctuation
    words = [word.lower() for word in words if word.isalnum()]

    for word in words:
        if word in vector:
            vector[word] += 1
        else:
             vector[word] = 1
    return vector

Przykładowy wektor:

In [7]:
create_vector('moby_dick_pt_10.txt',dictionary)

{'the': 17,
 'project': 0,
 'gutenberg': 0,
 'ebook': 0,
 'of': 14,
 'moby': 0,
 'dick': 0,
 'or': 3,
 'whale': 3,
 'by': 2,
 'herman': 0,
 'melville': 0,
 'this': 4,
 'is': 2,
 'for': 3,
 'use': 0,
 'anyone': 0,
 'anywhere': 0,
 'at': 0,
 'no': 0,
 'cost': 0,
 'and': 6,
 'with': 1,
 'almost': 0,
 'restrictions': 0,
 'whatsoever': 0,
 'you': 0,
 'may': 0,
 'copy': 0,
 'it': 1,
 'give': 0,
 'away': 0,
 'under': 0,
 'terms': 0,
 'license': 0,
 'included': 0,
 'online': 0,
 'title': 0,
 'author': 0,
 'release': 0,
 'date': 0,
 'december': 0,
 '25': 0,
 '2008': 0,
 '2701': 0,
 'last': 0,
 'updated': 0,
 '3': 0,
 '2017': 0,
 'language': 0,
 'english': 0,
 'character': 0,
 'set': 0,
 'encoding': 0,
 'start': 0,
 'produced': 0,
 'daniel': 0,
 'lazarus': 0,
 'jonesey': 0,
 'david': 0,
 'widger': 0,
 'contents': 0,
 'etymology': 0,
 'extracts': 0,
 'supplied': 0,
 'a': 5,
 'chapter': 0,
 '1': 0,
 'loomings': 0,
 '2': 0,
 '4': 0,
 'counterpane': 0,
 '5': 0,
 'breakfast': 0,
 '6': 0,
 'street': 0

### 4.
Zbuduj rzadką macierz wektorów cech term-by-document matrix w której wektory cech ułożone są kolumnowo Am×n = [d1|d2| . . . |dn] (m jest liczbą termów w
słowniku, a n liczbą dokumentów)


In [8]:
def create_sparse(files, dictionary):
    sparse = []
    for file in files:
        vector = create_vector(file, dictionary)
        sparse.append(vector)
    return sparse

In [9]:
matrix = create_sparse(files, dictionary)

### 5.
 Przetwórz wstępnie otrzymany zbiór danych mnożąc elementy bag-of-words przez
inverse document frequency. Operacja ta pozwoli na redukcję znaczenia często występujących słów.


In [10]:
def get_nw(word, sparse):
    counter = 0
    for row in sparse:
        if row.get(word) != 0:
            counter += 1
    return counter

In [11]:
def multiply_by_IDF(dictionary, sparse):
    for word in dictionary:
        nw = get_nw(word, sparse)
        if nw == 0:
            idf = 0
        else:
            idf = float(math.log(len(matrix)/nw))
        for row in sparse:
            row[word] = row.get(word) * idf

In [12]:
multiply_by_IDF(dictionary, matrix)

### 6.
Napisz program pozwalający na wprowadzenie zapytania (w postaci sekwencji
słów) przekształcanego następnie do reprezentacji wektorowej q (bag-of-words).
Program ma zwrócić k dokumentów najbardziej zbliżonych do podanego zapytania q. Użyj korelacji między wektorami jako miary podobieństwa

In [13]:
def create_vector_from_text(text, dictionary):
    vector = {}

    for word in dictionary:
        vector[word] = 0
    
    words = nltk.word_tokenize(text)
    
    #remove all punctuation
    words = [word.lower() for word in words if word.isalnum()]
    
    for word in words: 
        if word in vector:
            vector[word] += 1
    return list(vector.values())

In [14]:
def get_closest_documents(text, k, dictionary, sparse):
    text_vector = create_vector_from_text(text, dictionary)
    results = []
    
    for idx, row in enumerate(sparse):
        cos = np.matmul(np.array(text_vector).T, np.array(list(row.values()))) / (np.linalg.norm(text_vector) * np.linalg.norm(list(row.values())))
        if len(results)  < k:
            results.append((idx, cos))
        else:
            if cos > results[0][1] or math.isnan(results[0][1]):
                results[0] = (idx, cos)
        results = sorted(results, key = lambda x : x[1], reverse=True)
    return results  

In [15]:
text = "“Scarcely had we proceeded two days on the sea, when about sunrise a great many Whales and other monsters of the sea, appeared. Among the former, one was of a most monstrous size.... This came towards us, open-mouthed, raising the waves on all sides, and beating the sea before him into a foam.” —_Tooke’s Lucian_. “_The True History"
results = get_closest_documents(text, 5, dictionary, matrix)

In [37]:
results

[(2, 0.26183941261482957),
 (4, 0.045440766596732),
 (3, 0.036436620766064916),
 (0, 0.0024059993034086366),
 (1, 0.001281909599287602)]

Wynik wyszukania jest poprawny, pierwszy znaleziony plik jest tym z którego pochodzi fragment tekstu.

### 7. 
Zastosuj normalizację wektorów cech dj i wektora q, tak aby miały one długość 1.
Użyj zmodyfikowanej miary podobieństwa otrzymując


In [17]:
from sklearn.preprocessing import normalize

In [18]:
def normalize_matrix(matrix):
    results = []
    for i, row in enumerate(matrix):
        results.append(normalize([list(row.values())], norm="l1"))
    return results

In [20]:
normal_matrix = normalize_matrix(matrix)

In [24]:
def normalize_vector(text, dictionary):
    normalized_vector = normalize([create_vector_from_text(text, dictionary)],norm="l1")
    results = []
    for i, row in enumerate(normal_matrix):
        cos = np.matmul(np.array(normalized_vector[0]).T,np.array(row[0]))/(np.linalg.norm(normalized_vector)*np.linalg.norm(row))
        results.append((i, cos))
    return results

In [34]:
def get_closest_documents_normalized(text, k, dictionary):
    cos_vector = normalize_vector(text, dictionary)
    cos_vector = sorted(cos_vector,key=lambda x: x[1], reverse=True)
    return cos_vector[:k]

In [35]:
normalized_result = get_closest_documents_normalized(text, 5, dictionary)

In [36]:
print(normalized_result)

[(2, 0.2618394126148296), (271, 0.0913418953770854), (476, 0.09090090693396023), (121, 0.08856100190466853), (169, 0.08075602643879543)]


Na pierwszym miejscu znowu pojawia się poprawny wynik, reszta wyników jest inna niż podczas wyszukiwania bez normalizacji

### 8.
W celu usunięcia szumu z macierzy A (normalized_matrix) zastosuj SVD i low rank approximation

In [38]:
import scipy

In [53]:
def svd_and_low_rank(k):
    A = []
    for row in normal_matrix:
        A.append(row[0])
    u, s, vt = scipy.sparse.linalg.svds(np.array(A), k=k)
    return u @ np.diag(s) @ vt

In [69]:
def normalize_vectors_svd(text, k, dictionary):
    result = svd_and_low_rank(k)
    vector_normalized = normalize([create_vector_from_text(text, dictionary)],norm="l1")
    results = []
    for i, row in enumerate(result):
        cos = np.matmul(np.array(vector_normalized[0]).T,np.array(row))/(np.linalg.norm(vector_normalized)*np.linalg.norm(row))
        results.append((i, cos))
    return results

In [75]:
def get_closest_documents_svd(text, k, approx_k, dictionary):
    vectors = normalize_vectors_svd(text, approx_k, dictionary)
    vectors = sorted(vectors, key=lambda x: x[1], reverse=True)
    return vectors[:k]

In [72]:
svd_result = get_closest_documents_svd(text, 3, 150, dictionary)

In [73]:
print(svd_result)

[(2, 0.19775867082380671), (272, 0.19565453956129214), (270, 0.18421393274061776)]


Po usunięciu szumu wyniki z wyjątkiem pierwszego zmieniły się.

### 9. 
Porównaj działanie programu bez usuwania szumu i z usuwaniem szumu. Dla jakiej wartości k wyniki wyszukiwania są najlepsze (subiektywnie). Zbadaj wpływ
przekształcenia IDF na wyniki wyszukiwania.


Bez odszumiania:

In [74]:
print(normalized_result)

[(2, 0.2618394126148296), (271, 0.0913418953770854), (476, 0.09090090693396023), (121, 0.08856100190466853), (169, 0.08075602643879543)]


Z odszumianiem:

In [76]:
for k in range(5,500,50):
    print("k = ",k)
    print(get_closest_documents_svd(text, 3, k, dictionary))

k =  5
[(95, 0.1835704530909705), (361, 0.18351963822253747), (102, 0.18349224332568406)]
k =  55
[(104, 0.19491507374623), (270, 0.1938096185897583), (412, 0.19064561643560102)]
k =  105
[(272, 0.19818032685245535), (270, 0.19213667721713745), (2, 0.19010093646985907)]
k =  155
[(2, 0.19868172487374658), (272, 0.19300849137239195), (270, 0.18505428309683472)]
k =  205
[(2, 0.22090516790822062), (270, 0.17880071961691596), (128, 0.17522969406947023)]
k =  255
[(2, 0.22693811838555875), (128, 0.16927521156466918), (271, 0.16428984334488336)]
k =  305
[(2, 0.24457066232968025), (128, 0.16606258090746112), (476, 0.15680469258794957)]
k =  355
[(2, 0.24877768434056236), (128, 0.162665909397546), (476, 0.1543960926013011)]
k =  405
[(2, 0.25838356306697474), (128, 0.15602631904678074), (476, 0.14805037625530715)]
k =  455
[(2, 0.26902426785496947), (476, 0.14904123933726782), (99, 0.14321389824918246)]


Dla dla k >= 155 zaczynami otrzymać poprawny wynik

### Wnioski:
Wyniki z odszumianiem są bardziej trafne niż bez odszumiania, jednak koszt obliczania SVD dla dużych macierzy może być odczuwalny. Przekształcenie IDF pozwala na redukcję znaczenia słów, które występują wielokrotnie w dokumentach, przez co wyniki są bardziej konkretne.